In [33]:
import numpy as np
import scipy.optimize as opt
import cvxpy as cvx
import math as math
import matplotlib.pyplot as plt
from numpy.linalg import norm

### 1.0 Setting up the objective function for equation (7)

Explanation: this implementation treats W matrices as completely exogenous and given. W_ls is a 1D array of length T+1 of n-by-n matrix. 

In [34]:
def QMLE_obj(sigma,lam,gamma, rho, beta):
    return -1/2*((n-1)*T)*np.log(2*np.pi)-1/2*((n-1)*T)*np.log(sigma**2)-T*np.log(1-lam)+np.sum(get_S(sigma,lam,gamma, rho, beta))-1/(2*sigma**2)*np.sum(get_V(sigma,lam,gamma, rho, beta))

In [35]:
def get_S(sigma,lam,gamma, rho, beta):
    S_ls = []
    for i in range(T):
        W = W_ls[i+1]
        S_nt = np.identity(n) - lam*W
        # further impose that the determinant is positive to allow logarithm to work
        det_S = np.linalg.det(S_nt)
        S_ls.append(np.log(abs(det_S)))
    return S_ls

In [36]:
def get_V(sigma,lam,gamma, rho, beta):
    V_ls = []
    for i in range(T):
        W = W_ls[i+1]
        S_nt = np.identity(n) - lam*W
        SY_ls = []
        for j in range(T):
            W2 = W_ls[j+1]
            S_nt2 = np.identity(n) - lam*W2
            SY_ls.append(np.matmul(S_nt2,np.array(y[j+1]).reshape(n,1)))
        SY_sum = sum(SY_ls)
        # first component
        SY_tilde = (np.matmul(S_nt,np.array(y[j+1]).reshape(n,1))-(1/T)*SY_sum).reshape(n,1)
        # second component
        Y_ls = []
        for k in range(T):
            Y_ls.append(y[k])
        Y_tilde_lag = np.array(y[i]-(1/T)*sum(Y_ls)).reshape(n,1)
        WY_ls = []
        for l in range(T):
            W3 = W_ls[l]
            WY_ls.append(np.matmul(W3,np.array(y[l]).reshape(n,1)))
        WY_sum = sum(WY_ls)
        W_lag = W_ls[i]
        WY_tilde_lag = np.matmul(W_lag, np.array(y[i]).reshape(n,1))-(1/T)*(WY_sum)
        X_ls = []
        for m in range(T):
            X_ls.append(x[m])
        X_sum = sum(X_ls)
        X_tilde = np.array(x[i]-(1/T)*X_sum).reshape(n,1)
        Z_tilde = [Y_tilde_lag,WY_tilde_lag,X_tilde]
        # third component
        l_n = np.ones(n).reshape(n,1)
        alpha_ln = alpha[i]*l_n
        # forth component
        J_n = np.identity(n)-(1/n)*np.matmul(l_n,l_n.transpose())
        # fifth component
        V_tilde = np.array(SY_tilde-(gamma*Z_tilde[0]+rho*Z_tilde[1]+beta*Z_tilde[2])).reshape(n,1)
        # putting everything together
        VJ = np.matmul(V_tilde.transpose(),J_n)
        VJV = np.matmul(VJ,V_tilde)
        V_ls.append(VJV)     
    return V_ls

### 1.1 Test if objective function works

In [37]:
import random as random

In [38]:
def is_even(num):
    if (num % 2) == 0:
        return True
    else:
        return False

In [39]:
# let's first do the case when beta is a constant
n = 4
T = 4
alpha = np.random.normal(0,1,4).transpose()
c0 = np.random.normal(0,1,4).transpose()
x = []
for i in range(T):
    x.append(np.random.normal(0,1,4).transpose())
y = []
for i in range(T+1):
    y.append(np.random.normal(0,1,4).transpose())
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0]]).reshape(4,4)
w0

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0]])

In [40]:
# manually setting up a row-normalized spatial weight vector with 0 diagonals for odd time points
w1 = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[1,0,0,0]]).reshape(4,4)
w1

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0]])

In [41]:
# manually setting up the length T+1 weight matrix vector list
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)
W_ls

[array([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]]),
 array([[0, 0, 1, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]]),
 array([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]]),
 array([[0, 0, 1, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]]),
 array([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])]

In [42]:
# setting up parameter plug-in values
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = 1

V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,sigma,n).transpose())

In [43]:
QMLE_obj(sigma,lam,gamma,rho,beta)

-17.36473677084214

In [44]:
# works.

### 2.1 Implement Optimization

#### 2.1.1 Scipy.optimize

In [45]:
from scipy.optimize import minimize

In [46]:
def QMLE_obj_scipy(params):
    sigma, lam, gamma, rho, beta = params
    obj= -1/2*((n-1)*T)*np.log(2*np.pi)-1/2*((n-1)*T)*np.log(sigma**2)-T*np.log(1-lam)+np.sum(get_S(sigma,lam,gamma, rho, beta))-1/(2*sigma**2)*np.sum(get_V(sigma,lam,gamma, rho, beta))
    #maximize is to minimize the negative
    return (-1)*obj

In [47]:
def scipy_constraint(T):
    con0 = lambda params: params[0]
    lc0 = NonlinearConstraint(con0, 0, np.inf)
    con1 = lambda params: -params[1]+1
    lc1 = NonlinearConstraint(con0, 0, np.inf)
    my_con = [lc0,lc1]
    for i in range(T):
        temp = lambda params: np.linalg.det(np.identity(n)-params[1]*W_ls[i+1])
        nlc = NonlinearConstraint(temp, 0, np.inf)
        my_con.append(nlc)
    return my_con

In [48]:
def QMLE_scipy_estimate(n_0,T_0, alpha_0, c0, x_dataset, y_attribute, Weight_ls, initial_guess, constrain = True):
    '''
    alpha_0 is a T*1 vector of time effect
    x_dataset is a list of length T of n*1 vectors (we assume only 1 feature so beta is constant) 
    y_attribute is a n*(T+1) matrix with y[0] known
    c_0 is a N*1 column vector of individual effects  
    initial_guess is 1D np.array of parameters described in order of the objective function above
    '''
    global n
    n = n_0
    global T
    T = T_0
    global x
    x = x_dataset
    global y
    y = y_attribute
    global c
    c = c0
    global alpha
    alpha = alpha_0
    global W_ls
    W_ls = Weight_ls
    # implement the constrained version
    # set inequality constraints on sigma to be greater than 0
    if constrain == True:
        const = scipy_constraint(T)
        res = minimize(QMLE_obj_scipy, initial_guess, method='trust-constr', constraints = const)
        # return maximizing parameters 
        return res.x
    else:
        res = minimize(QMLE_obj_scipy, initial_guess, method='nelder-mead', options={'xatol': 0.000001, 'disp': False})
        return res.x

#### 2.1.1.1 Test

In [49]:
from scipy.optimize import NonlinearConstraint
from scipy.optimize import LinearConstraint

In [50]:
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = 1
initial_guess = [sigma, lam, gamma, rho, beta]
initial_guess

[1, 0.5, 0.1, 0.2, 1]

In [52]:
# unconstrained
res = minimize(QMLE_obj_scipy, initial_guess, method='nelder-mead', options={'xatol': 0.001, 'disp': True})

Optimization terminated successfully.
         Current function value: 8.355322
         Iterations: 278
         Function evaluations: 449


In [53]:
res.x

array([ 0.4249679 , -0.40651091,  0.02580851, -0.02193079,  0.04423072])

In [51]:
# unconstrained
res = minimize(QMLE_obj_scipy, initial_guess, method='Powell')

/var/folders/44/8_54q8w53q34chh_wjy62nyw0000gn/T/ipykernel_86366/2865426405.py:3: RuntimeWarning: invalid value encountered in log
  obj= -1/2*((n-1)*T)*np.log(2*np.pi)-1/2*((n-1)*T)*np.log(sigma**2)-T*np.log(1-lam)+np.sum(get_S(sigma,lam,gamma, rho, beta))-1/(2*sigma**2)*np.sum(get_V(sigma,lam,gamma, rho, beta))


In [21]:
con0 = lambda params: params[0]
lc0 = NonlinearConstraint(con0, 0, np.inf)
con1 = lambda params: -params[1]+1
lc1 = NonlinearConstraint(con0, 0, np.inf)

In [22]:
con2 = lambda params: np.linalg.det(np.identity(n)-params[1]*W_ls[1])
nlc1 = NonlinearConstraint(con2, 0, np.inf)
con3 = lambda params: np.linalg.det(np.identity(n)-params[1]*W_ls[2])
nlc2 = NonlinearConstraint(con3, 0, np.inf)
con4 = lambda params: np.linalg.det(np.identity(n)-params[1]*W_ls[3])
nlc3 = NonlinearConstraint(con4, 0, np.inf)
con5 = lambda params: np.linalg.det(np.identity(n)-params[1]*W_ls[4])
nlc4 = NonlinearConstraint(con5, 0, np.inf)

In [23]:
constraints = [lc0,lc1,nlc1,nlc2,nlc3,nlc4]
constraints

In [24]:
res = minimize(QMLE_obj_scipy, initial_guess, method='trust-constr', constraints = [lc0,lc1,nlc1,nlc2,nlc3,nlc4])

/Users/helangchun/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [25]:
res

 barrier_parameter: 1.0240000000000006e-08
 barrier_tolerance: 1.0240000000000006e-08
          cg_niter: 168
      cg_stop_cond: 0
            constr: [array([0.80024561]), array([0.80024561]), array([0.08190539]), array([0.05537748]), array([0.08190539]), array([0.05537748])]
       constr_nfev: [390, 390, 390, 390, 390, 390]
       constr_nhev: [0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.3512239456176758
               fun: 10.854415618042317
              grad: array([ 0.00000000e+00, -6.55651093e-06, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00])
               jac: [array([[ 1.,  0., -0., -0.,  0.]]), array([[ 1.,  0., -0., -0.,  0.]]), array([[ 0.        , -2.83386759, -0.        , -0.        ,  0.        ]]), array([[ 0.        , -1.94383388, -0.        , -0.        ,  0.        ]]), array([[ 0.        , -2.83386759, -0.        , -0.        ,  0.        ]]), array([[ 0.        , -1.9

In [26]:
res.x

array([ 0.80024561,  0.97191693, -0.00757313, -0.0424243 ,  0.00265133])

In [27]:
# works. 

In [28]:
# check if the function works
initial_guess = [sigma, lam, gamma, rho, beta]
QMLE_scipy_estimate(n,T, alpha, c0, x, y, W_ls, initial_guess)

array([ 0.80024553,  0.97191649, -0.00757311, -0.04242426,  0.00265133])

In [29]:
# works.

### 3.1 Implement the final test

In [36]:
# first, generate samples

In [37]:
def generate_weight_matrix(n):
    row = []
    for i in range(n):
        temp_row = np.zeros(n)
        # ensure that diagonal is nonzero and that the row is normalized
        no_stop = True
        while no_stop:
            place_one = random.randint(0,n-1)
            if place_one != i:
                no_stop = False
        temp_row[place_one] = 1
        row.append(temp_row.tolist())
    w = np.array(row).reshape(n,n)
    return w

In [38]:
# generate samples
t_theta = [0.1,0.2,1,0.5,1]
t_gamma, t_rho, t_beta, t_lam, t_sig = t_theta
n = 49
T = 10
alpha = np.random.normal(0,1,T).transpose()
c0 = np.random.normal(0,1,n).transpose()
x = []
for i in range(T):
    x.append(np.random.normal(0,1,n).transpose())
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = generate_weight_matrix(n)
w0

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
w1 = generate_weight_matrix(n)
w1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

In [41]:
V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,t_sig,n).transpose())

In [42]:
Y0 = np.random.normal(0,1,n).transpose()
Y0

array([-0.21116716,  1.61031442, -1.64608597, -0.35904946, -1.26374641,
        1.55624187, -0.41252036, -0.49702865,  0.20940203, -0.46467785,
        0.47561019, -0.72171455, -0.03665072,  0.05310561,  2.01261466,
        0.78820414, -0.33759127, -0.62228505, -0.21490303,  1.60393414,
       -0.29662607,  1.28268814, -1.67929753, -1.11989835, -0.99737594,
       -0.31232823, -1.11051094,  0.12256137,  1.90113063, -1.32972554,
        0.23160958, -0.46194964,  1.87574274, -0.9269115 ,  0.02253627,
        0.5453005 ,  1.6002788 , -1.12214175, -1.66581805, -0.66887738,
       -0.33189773,  0.33403033, -2.00630849,  0.02561084, -0.35334656,
       -0.53646744, -1.05355226, -0.40771399, -0.05335904])

In [43]:
Y_ls = []
Y_ls.append(Y0)
for i in range(T):
    l_n = np.ones(n).transpose()
    c_vec = t_gamma*Y_ls[i]+t_rho*np.matmul(W_ls[i],Y_ls[i])+t_beta*x[i]+c0+alpha[i]*l_n+V_nt[i]
    Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec)
    Y_ls.append(Y_nt)

In [44]:
# initial guess (set off but not by much)
initial_guess = [1.5,0.2,0.5,0.3,1.2]

In [45]:
QMLE_scipy_estimate(n,T, alpha, c0, x, Y_ls, W_ls, initial_guess)

array([ 1.60287427e+00,  3.07764349e-01, -3.64257110e-03,  5.66121436e-02,
        1.88889252e-04])

In [46]:
QMLE_scipy_estimate(n,T, alpha, c0, x, Y_ls, W_ls, initial_guess, constrain = False)

Optimization terminated successfully.
         Current function value: 904.072687
         Iterations: 275
         Function evaluations: 459


array([ 1.60287419e+00,  3.07764381e-01, -3.64273661e-03,  5.66121262e-02,
        1.88525486e-04])

In [47]:
t_theta = [t_sig,t_lam,t_gamma,t_rho,t_beta]
t_theta

[1, 0.5, 0.1, 0.2, 1]

### 3.2 Performance Stats

In [48]:
from tabulate import tabulate
from sklearn.metrics import mean_squared_error
from math import sqrt

In [49]:
# obtain 50 QMLE_estimates from 50 monte-carlo experiments without fixing weight matrices for each experiment.

def five_hun_samples_scipy(n,T,t_theta,g_theta, N=500, constrain=True):
    # set up true and guess parameters
    t_sig, t_lam, t_gamma, t_rho, t_beta = t_theta
    g_sig, g_lam, g_gamma, g_rho, g_beta = g_theta
    initial_guess = [g_sig, g_lam, g_gamma, g_rho, g_beta]
    sample_ls = []
    for k in range(N):
        # set up samples
        alpha = np.random.normal(0,1,T).transpose()
        alpha = np.nan_to_num(alpha)
        alpha[alpha == -np.inf] = 0
        alpha[alpha == np.inf] = 0
        c0 = np.random.normal(0,1,n).transpose()
        c0 = np.nan_to_num(c0)
        c0[c0 == -np.inf] = 0
        c0[c0 == np.inf] = 0
        x = []
        for i in range(T):
            tem = np.random.normal(0,1,n).transpose()
            tem = np.nan_to_num(tem)
            tem[tem == -np.inf] = 0
            tem[tem == np.inf]=0
            x.append(tem)
        # manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
        w0 = generate_weight_matrix(n)
        w1 = generate_weight_matrix(n)
        W_ls = []
        for i in range(T+1):
            if is_even(i):
                W_ls.append(w0)
            else:
                W_ls.append(w1)
        V_nt = []
        for i in range(T):
            tem = np.random.normal(0,t_sig,n).transpose()
            tem = np.nan_to_num(tem)
            tem[tem == -np.inf] = 0
            tem[tem == np.inf]=0
            V_nt.append(tem)
        Y0 = np.random.normal(0,1,n).transpose()
        Y_ls = []
        Y_ls.append(Y0)
        for i in range(T):
            l_n = np.ones(n).transpose()
            c_vec = t_gamma*Y_ls[i]+t_rho*np.matmul(W_ls[i],Y_ls[i])+t_beta*x[i]+c0+alpha[i]*l_n+V_nt[i]
            Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec)
            Y_ls.append(Y_nt)
        # run scipy optimize
        sample = QMLE_scipy_estimate(n,T, alpha, c0, x, Y_ls, W_ls, initial_guess, constrain)
        sample_ls.append(sample)
    # return sample list
    return sample_ls

First, run 50 experiments.

In [121]:
sample_ls = five_hun_samples_scipy(n=49,T=10,t_theta=[1, 0.5, 0.1, 0.2, 1],g_theta= [1.5,0.2,0.5,0.3,1.2], N=50, constrain=True)

/Users/helangchun/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [95]:
def obtain_table_stats(sample_ls,t_theta):
    t_sig,t_lam,t_gamma,t_rho,t_beta = t_theta
    gam_ls = []
    rho_ls = []
    beta_ls = []
    lam_ls = []
    sig_ls = []
    n = len(sample_ls)
    for i in range(n):
        sig_ls.append(sample_ls[i][0])
        lam_ls.append(sample_ls[i][1])
        gam_ls.append(sample_ls[i][2])
        rho_ls.append(sample_ls[i][3])
        beta_ls.append(sample_ls[i][4])
    # bias
    gam_bias = np.array(gam_ls).mean()-t_gamma
    rho_bias = np.array(rho_ls).mean()-t_rho
    beta_bias = np.array(beta_ls).mean()-t_beta
    lam_bias = np.array(lam_ls).mean()-t_lam
    sig_bias = np.array(sig_ls).mean()-t_sig
    bias = [gam_bias, rho_bias, beta_bias,lam_bias, sig_bias]
    
    # rmse
    gam_rmse = sqrt(mean_squared_error(np.array([t_gamma]*n), np.array(gam_ls)))
    rho_rmse = sqrt(mean_squared_error(np.array([t_rho]*n), np.array(rho_ls)))
    beta_rmse = sqrt(mean_squared_error(np.array([t_beta]*n), np.array(beta_ls)))
    lam_rmse = sqrt(mean_squared_error(np.array([t_lam]*n), np.array(lam_ls)))
    sig_rmse = sqrt(mean_squared_error(np.array([t_sig]*n), np.array(sig_ls)))
    rmse = [gam_rmse, rho_rmse, beta_rmse,lam_rmse, sig_rmse]
    
    # sd
    gam_sd = np.array(gam_ls).std()
    rho_sd = np.array(rho_ls).std()
    lam_sd = np.array(lam_ls).std()
    sig_sd = np.array(sig_ls).std()
    beta_sd = np.array(beta_ls).std()
    sd = [gam_sd, rho_sd, beta_sd,lam_sd, sig_sd]
    
    # print table
    bias_ls = ["Bias", gam_bias, rho_bias, beta_bias, lam_bias, sig_bias]
    sd_ls = ["SD", gam_sd, rho_sd, beta_sd, lam_sd, sig_sd]
    rmse_ls = ["RMSE", gam_rmse, rho_rmse, beta_rmse, lam_rmse, sig_rmse]
    data = [bias_ls, sd_ls, rmse_ls]
    #define header names
    col_names = ["Measures (n={}; T={}; theta={})".format(n, T, t_theta), "gamma", "rho", "beta", "lambda", "sigma"]
    # display table
    print(tabulate(data, headers=col_names))

#### 3.1.1 table 1 description
1. (n,T,N) = (49,10,50)
<br>
2. t_theta = [t_sig,t_lam,t_gamma,t_rho,t_beta] = [1, 0.5, 0.1, 0.2, 1]
<br>
3. initial_guess = [1.5,0.2,0.5,0.3,1.2]
<br>
4. Other given vectors are generated by iid standard normal distribution.
<br>
5. Weight matrices alternate on odd and even time points. For each monte carlo trial, they are generated iid. 
<br>
6. Optimization method is trust-region from scipy.minimize with constraints.

In [96]:
obtain_table_stats(sample_ls,t_theta)

Measures (n=50; T=10; theta=[1, 0.5, 0.1, 0.2, 1])        gamma         rho        beta      lambda     sigma
----------------------------------------------------  ---------  ----------  ----------  ----------  --------
Bias                                                  -0.10905   -0.154952   -0.99703    -0.303485   0.543793
SD                                                     0.011966   0.0231284   0.0199779   0.0965275  0.188202
RMSE                                                   0.109705   0.156668    0.99723     0.318466   0.57544


#### 3.1.2 table 2 decription

1. (n,T) = (49,10,50)
<br>
2. t_theta = [t_sig,t_lam,t_gamma,t_rho,t_beta] = [1, 0.5, 0.1, 0.2, 1]
<br>
3. initial_guess = [1.5,0.2,0.5,0.3,1.2]
<br>
4. Other given vectors are generated by iid standard normal distribution.
<br>
5. Weight matrices alternate on odd and even time points. For each monte carlo trial, they are generated iid. 
<br>
6. Optimization method is nelder-mead without constraints.

In [138]:
%%capture
sample_ls = five_hun_samples_scipy(n=49,T=10,t_theta=[1, 0.5, 0.1, 0.2, 1],g_theta= [1.5,0.2,0.5,0.3,1.2], N=50, constrain=False)

In [139]:
obtain_table_stats(sample_ls,t_theta)

Measures          gamma         rho        beta      lambda     sigma
----------  -----------  ----------  ----------  ----------  --------
Bias        -0.107543    -0.155652   -1.00298    -0.305689   0.528854
SD           0.00891143   0.0292382   0.0125355   0.0880344  0.210978
RMSE         0.107912     0.158374    1.00306     0.318113   0.569384


#### 3.1.3 table 3 description
1. (n,T,N) = (49,10,200)
<br>
2. t_theta = [t_sig,t_lam,t_gamma,t_rho,t_beta] = [1, 0.5, 0.1, 0.2, 1]
<br>
3. initial_guess = [1.5,0.2,0.5,0.3,1.2]
<br>
4. Other given vectors are generated by iid standard normal distribution.
<br>
5. Weight matrices alternate on odd and even time points. For each monte carlo trial, they are generated iid. 
<br>
6. Optimization method is nelder-mead without constraints.

In [142]:
%%capture
sample_ls = five_hun_samples_scipy(n=49,T=10,t_theta=[1, 0.5, 0.1, 0.2, 1],g_theta= [1.5,0.2,0.5,0.3,1.2], N=200, constrain = False)

In [143]:
obtain_table_stats(sample_ls,t_theta)

Measures         gamma         rho        beta      lambda     sigma
----------  ----------  ----------  ----------  ----------  --------
Bias        -0.10802    -0.150984   -1.00084    -0.312912   0.573106
SD           0.0107432   0.0296407   0.0138838   0.0942078  0.197327
RMSE         0.108553    0.153866    1.00094     0.326786   0.606126


### 3.3 Run samples and take the sample mean for fixed weight matrix

In [56]:
# generate weight list

def generate_wls(n,T, alternate = True):
    w0 = generate_weight_matrix(n)
    w1 = generate_weight_matrix(n)
    W_ls = []
    if alternate:
        for i in range(T+1):
            if is_even(i):
                W_ls.append(w0)
            else:
                W_ls.append(w1) 
    else:
        for i in range(T+1):
            W_ls.append(w0)
    return W_ls        

In [97]:
# obtain five hundrend QMLE_estimates from 500 monte-carlo experiments

def generate_samples_scipy_fix_weight(n,T,t_theta,g_theta,W_ls, N=50, constrain=True):
    # set up true and guess parameters
    t_sig, t_lam, t_gamma, t_rho, t_beta = t_theta
    g_sig, g_lam, g_gamma, g_rho, g_beta = g_theta
    # set up initial guess
    initial_guess = [g_sig, g_lam, g_gamma, g_rho, g_beta]
    # sample list
    sample_ls = []
    for k in range(N):
        # set up samples
        alpha = np.random.normal(0,1,T).transpose()
        alpha = np.nan_to_num(alpha)
        alpha[alpha == -np.inf] = 0
        alpha[alpha == np.inf] = 0
        c0 = np.random.normal(0,1,n).transpose()
        c0 = np.nan_to_num(c0)
        c0[c0 == -np.inf] = 0
        c0[c0 == np.inf] = 0
        x = []
        for i in range(T):
            tem = np.random.normal(0,1,n).transpose()
            tem = np.nan_to_num(tem)
            tem[tem == -np.inf] = 0
            tem[tem == np.inf]=0
            x.append(tem)
        V_nt = []
        for i in range(T):
            tem = np.random.normal(0,t_sig,n).transpose()
            tem = np.nan_to_num(tem)
            tem[tem == -np.inf] = 0
            tem[tem == np.inf]=0
            V_nt.append(tem)
        Y0 = np.random.normal(0,1,n).transpose()
        Y_ls = []
        Y_ls.append(Y0)
        for i in range(T):
            l_n = np.ones(n).transpose()
            c_vec = t_gamma*Y_ls[i]+t_rho*np.matmul(W_ls[i],Y_ls[i])+t_beta*x[i]+c0+alpha[i]*l_n+V_nt[i]
            Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec)
            Y_ls.append(Y_nt)
        # run scipy optimize
        sample = QMLE_scipy_estimate(n,T, alpha, c0, x, Y_ls, W_ls, initial_guess, constrain)
        sample_ls.append(sample)
    # return sample list
    return sample_ls

In [69]:
W_ls = generate_wls(49,10, alternate = True)
[W_ls[0],W_ls[1]]

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])]

In [66]:
t_theta=[1, 0.5, 0.1, 0.2, 1]
t_sig, t_lam, t_gamma, t_rho, t_beta = t_theta

In [68]:
sample_ls = generate_samples_scipy_fix_weight(n=49,T=10,t_theta=[1, 0.5, 0.1, 0.2, 1],g_theta= [1.5,0.2,0.5,0.3,1.2], N=50, W_ls = W_ls, constrain=True)

In [70]:
obtain_table_stats(sample_ls,t_theta)

Measures        gamma         rho        beta      lambda     sigma
----------  ---------  ----------  ----------  ----------  --------
Bias        -0.10905   -0.154952   -0.99703    -0.303485   0.543793
SD           0.011966   0.0231284   0.0199779   0.0965275  0.188202
RMSE         0.109705   0.156668    0.99723     0.318466   0.57544


In [85]:
# generate multiple table (each table is an output of a monte carlo experiement of N samples)
def generate_multi_table(n_ls,T_ls,t_theta_ls,g_theta_ls,W_ls, N=50, auto_weight = True, alternate = True, constrain=True):
    num = len(n_ls)
    total_W_ls = []
    for i in range(num):
        if auto_weight:
            W_ls = generate_wls(n_ls[i],T_ls[i], alternate = alternate)
            total_W_ls.append(W_ls)
        sample_ls = generate_samples_scipy_fix_weight(n=n_ls[i],T=T_ls[i],t_theta=t_theta_ls[i],g_theta=g_theta_ls[i], N=N, W_ls = W_ls, constrain=constrain)
        # print tables
        obtain_table_stats(sample_ls,t_theta_ls[i])  
    if auto_weight:
        return total_W_ls

#### 3.2.1 Test generate_multi_table

In [98]:
n_ls = [50,50,50,50,50]
T_ls = [10,10,10,10,10]
t_theta_ls = [[1, 0.5, 0.1, 0.2, 1],[1, 0.5, 0.1, 0.2, 1],[1,0.2,0.4,0.2,1],[1, 0.5, 0.1, 0.2, 1],[1,0.2,0.4,0.2,1]]
g_theta_ls = [[1.5,0.2,0.5,0.3,1.2], [0.8,0.7,0.05,0.1,0.8], [1.1,0.25,0.45,0.35,1.1], [1.5, 1.5, 0.8, 1, 0.6],  [0.7,0.2,0.05,0.1,1.2]]

In [99]:
a=generate_multi_table(n_ls,T_ls,t_theta_ls,g_theta_ls,W_ls, N=50, auto_weight = True, alternate = True, constrain=False)

Measures (n=50; T=10; theta=[1, 0.5, 0.1, 0.2, 1])          gamma         rho        beta      lambda     sigma
----------------------------------------------------  -----------  ----------  ----------  ----------  --------
Bias                                                  -0.106008    -0.158756   -0.998531   -0.311192   0.519996
SD                                                     0.00855426   0.0226829   0.0122753   0.0957339  0.195872
RMSE                                                   0.106352     0.160368    0.998607    0.325585   0.555664
Measures (n=50; T=10; theta=[1, 0.5, 0.1, 0.2, 1])        gamma         rho        beta      lambda     sigma
----------------------------------------------------  ---------  ----------  ----------  ----------  --------
Bias                                                  -0.106603  -0.151044   -1.00293    -0.305985   0.623928
SD                                                     0.01078    0.0255777   0.0131525   0.0828427  0.187184


/var/folders/44/8_54q8w53q34chh_wjy62nyw0000gn/T/ipykernel_82092/2865426405.py:3: RuntimeWarning: invalid value encountered in log
  obj= -1/2*((n-1)*T)*np.log(2*np.pi)-1/2*((n-1)*T)*np.log(sigma**2)-T*np.log(1-lam)+np.sum(get_S(sigma,lam,gamma, rho, beta))-1/(2*sigma**2)*np.sum(get_V(sigma,lam,gamma, rho, beta))


Measures (n=50; T=10; theta=[1, 0.5, 0.1, 0.2, 1])          gamma    rho          beta    lambda    sigma
----------------------------------------------------  -----------  -----  ------------  --------  -------
Bias                                                  0.7            0.8  -0.4                 1      0.5
SD                                                    2.22045e-16    0     1.11022e-16         0      0
RMSE                                                  0.7            0.8   0.4                 1      0.5
Measures (n=50; T=10; theta=[1, 0.2, 0.4, 0.2, 1])          gamma         rho         beta      lambda     sigma
----------------------------------------------------  -----------  ----------  -----------  ----------  --------
Bias                                                  -0.407165    -0.154261   -0.999608    -0.0886742  0.533912
SD                                                     0.00672316   0.0316467   0.00827128   0.0744479  0.162124
RMSE                